In [1]:
# add path to explore_by_example src folder
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)


# import usual libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.metrics import f1_score

# our system imports
from aideme import *

%matplotlib inline

In [2]:
def plot_fscore(metrics):
    df_list = [pd.DataFrame.from_dict({i: metric for i, metric in enumerate(ls)}, orient='index') for ls in metrics]
    avg = sum([df['fscore'][~df['fscore'].isna()] for df in df_list]) / len(df_list)
    avg.plot(ylim=[0,1], marker='o')
    plt.show()

In [3]:
# DUMMY DATA
N = int(1e5)
dim = 2
limit = 2 * (0.001)**(1. / dim)
rng = np.random.RandomState(0)
X = rng.uniform(low=-2, high=2, size=(N, dim))  # do not forget to standardize the data. For this distribution, it should be fine without it.
y_subspace = np.vstack([np.abs(X[:, i]) < limit for i in range(dim)]).T.astype('float')  # partial labels (for each subspace)
y = y_subspace.min(axis=1)

index = -10 * np.arange(len(X))
labeled_set = LabeledSet(y, y_subspace, index)

# visualize data distribution
print('selectivity :', 100 * y.sum() / len(y), '%')

#plt.figure(figsize=(10,8))
#plt.scatter(X[:, 0], X[:, 1], s=0.5, c=['b' if lb else 'r' for lb in y])
#plt.show()

selectivity : 0.105 %


# NO FACTORIZATION EXAMPLE

In [4]:
# SET-UP EXPLORATION CONFIGURATION
NUMBER_OF_ITERATIONS = 100  # number of points to be labeled by the user
SUBSAMPLING = None
INITIAL_SAMPLER = stratified_sampler(labeled_set, pos=1, neg=1)  # start with one random positive sample and one random negative sample
#INITIAL_SAMPLER = random_sampler(10)
CALLBACK = [ # callback functions to be called at the end of each iteration
    classification_metrics(labeled_set.labels, 'fscore'), 
    three_set_metric,
]
CALLBACK_SKIP = 10
PRINT_CALLBACK_RESULT = True
CONVERGENCE_CRITERIA = [
    max_iter_reached(NUMBER_OF_ITERATIONS),
    #all_points_are_known,
    #metric_reached_threshold('fscore', 0.8),
    #metric_reached_threshold('tsm', 0.9),
]

explore = PoolBasedExploration(INITIAL_SAMPLER, SUBSAMPLING, CALLBACK, CALLBACK_SKIP, PRINT_CALLBACK_RESULT, CONVERGENCE_CRITERIA)

# CHOOSE AN ALGORITHM
#learner = RandomSampler(SVC(C=1e5, kernel='rbf', gamma='auto'))  # choose a random point
learner = SimpleMargin(C=1024, kernel='rbf')  # choose point closest to SVM decision boundary
#learner = KernelQueryByCommittee(kernel='rbf', sampling='deterministic', n_samples=8, warmup=1000, thin=100, rounding=True)  # version space algorithm
learner = DualSpaceModel(learner, sample_unknown_proba=0.5, mode='positive')  # Dual Space model


# RUN EXPLORATION
metrics = explore.run(X, labeled_set, learner, repeat=1)  # 'repeat' specifies how many times to repeat the exploration process

# COMPUTE AVERAGE F-SCORE OVER ALL REPEATS AND PLOT
#plot_fscore(metrics)
df = pd.DataFrame.from_dict({i: metric for i, metric in enumerate(metrics[0])}, orient='index')
df

iter: 1, fscore: 0.002886756660160009, tsm: 0.0
iter: 11, fscore: 0.02747971735147867, tsm: 0.0
iter: 21, fscore: 0.08177570093457943, tsm: 0.0
iter: 31, fscore: 0.35456110154905335, tsm: 0.0
iter: 41, fscore: 0.4251012145748988, tsm: 9.203300917262325e-05
iter: 51, fscore: 0.8586387434554974, tsm: 0.002589437819420784
iter: 61, fscore: 0.7589743589743589, tsm: 0.02013693113169553
iter: 71, fscore: 0.8471615720524017, tsm: 0.28936170212765955
iter: 81, fscore: 0.875, tsm: 0.46111111111111114
iter: 91, fscore: 0.9393939393939394, tsm: 0.6796875
iter: 101, fscore: 0.9705882352941176, tsm: 0.8521739130434782


,phase,iter_time,labeled_indexes,final_labels,partial_labels,fit_time,get_next_time,fscore,tsm,callback_time
0,begin,0.004101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,initial_sampling,0.039721,"[-579550, -449440]","[1.0, 0.0]","[[1.0, 1.0], [1.0, 0.0]]",0.001384,0.038337,0.002887,0.000000,0.462581
2,exploration,0.037270,[-483230],[0.0],"[[0.0, 0.0]]",0.001495,0.035775,NaN,NaN,NaN
3,exploration,0.037940,[-470170],[0.0],"[[0.0, 0.0]]",0.000740,0.037200,NaN,NaN,NaN
4,exploration,0.037486,[-125720],[0.0],"[[0.0, 0.0]]",0.000697,0.036789,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
97,exploration,0.130034,[-828540],[0.0],"[[1.0, 0.0]]",0.002372,0.127662,NaN,NaN,NaN
98,exploration,0.011294,[-334240],[1.0],"[[1.0, 1.0]]",0.009853,0.001441,NaN,NaN,NaN
99,exploration,0.395426,[-574460],[1.0],"[[1.0, 1.0]]",0.009769,0.385657,NaN,NaN,NaN
100,exploration,0.133982,[-537110],[0.0],"[[1.0, 0.0]]",0.002489,0.131493,NaN,NaN,NaN


# FACTORIZATION EXAMPLE

In [5]:
# SET-UP EXPLORATION CONFIGURATION
NUMBER_OF_ITERATIONS = 100  # number of points to be labeled by the user
SUBSAMPLING = None
INITIAL_SAMPLER = stratified_sampler(labeled_set, pos=1, neg=1)  # start with one random positive sample and one random negative sample
#INITIAL_SAMPLER = random_sampler(10)
CALLBACK = [ # callback functions to be called at the end of each iteration
    classification_metrics(labeled_set.labels, 'fscore'), 
    three_set_metric,
]
CALLBACK_SKIP = 10
PRINT_CALLBACK_RESULT = True
CONVERGENCE_CRITERIA = [
    max_iter_reached(NUMBER_OF_ITERATIONS),
    #metric_reached_threshold('fscore', 0.8),
    metric_reached_threshold('tsm', 1.0),
]

explore = PoolBasedExploration(INITIAL_SAMPLER, SUBSAMPLING, CALLBACK, CALLBACK_SKIP, PRINT_CALLBACK_RESULT, CONVERGENCE_CRITERIA)

# Factorized AL algorithms
PARTITION = [[i] for i in range(dim)]  #[[0], [1]]

# FACTORIZED VERSION SPACE
# label_function = 'AND', 'OR', 'PROD'
# loss = 'GREEDY', 'SQUARED', 'PRODUCT'
#learner = SubspatialVersionSpace(warmup=100, thin=10, n_samples=8, rounding=True, kernel='rbf', gamma=None, partition=PARTITION, label_function='AND', loss='GREEDY')

# FACTORIZED SIMPLE MARGIN
#learner = SubspatialSimpleMargin(C=1024, kernel='rbf', gamma=5, partition=PARTITION, label_function='AND')

# FACTORIZED DSM
learner = DualSpaceModel(SimpleMargin(C=1024, kernel='rbf'), mode='positive', sample_unknown_proba=0.5, partition=PARTITION)  # Dual Space model


# RUN EXPLORATION
metrics = explore.run(X, labeled_set, learner, repeat=1)  # use y_subspace here

# COMPUTE AVERAGE F-SCORE OVER ALL REPEATS AND PLOT
#plot_fscore(metrics)
df = pd.DataFrame.from_dict({i: metric for i, metric in enumerate(metrics[0])}, orient='index')
df

iter: 1, fscore: 0.0037672215843857636, tsm: 0.0
iter: 11, fscore: 0.2198952879581152, tsm: 0.0
iter: 21, fscore: 0.8125, tsm: 0.22419928825622776
iter: 31, fscore: 0.9454545454545454, tsm: 0.7948717948717948
iter: 41, fscore: 0.9805825242718447, tsm: 0.9439252336448598
iter: 51, fscore: 1.0, tsm: 1.0


,phase,iter_time,labeled_indexes,final_labels,partial_labels,fit_time,get_next_time,fscore,tsm,callback_time
0,begin,0.002083,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,initial_sampling,0.397519,"[-334240, -997150]","[1.0, 0.0]","[[1.0, 1.0], [0.0, 0.0]]",0.379171,0.018348,0.003767,0.000000,0.449461
2,exploration,0.266246,[-255330],[0.0],"[[0.0, 0.0]]",0.166832,0.099414,NaN,NaN,NaN
3,exploration,0.226990,[-588170],[0.0],"[[0.0, 0.0]]",0.220376,0.006614,NaN,NaN,NaN
4,exploration,0.126217,[-999100],[0.0],"[[0.0, 0.0]]",0.057346,0.068871,NaN,NaN,NaN
5,exploration,0.084914,[-700210],[0.0],"[[0.0, 0.0]]",0.045764,0.039150,NaN,NaN,NaN
6,exploration,0.006206,[-996030],[0.0],"[[0.0, 1.0]]",0.003369,0.002837,NaN,NaN,NaN
7,exploration,0.082266,[-857170],[0.0],"[[0.0, 1.0]]",0.011212,0.071054,NaN,NaN,NaN
8,exploration,0.047460,[-557460],[0.0],"[[0.0, 1.0]]",0.008808,0.038652,NaN,NaN,NaN
9,exploration,0.043878,[-583790],[0.0],"[[0.0, 0.0]]",0.042394,0.001484,NaN,NaN,NaN
